In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 15, 'lines.linewidth': 3, 'savefig.dpi': 1200})

import os, random, time, pickle, torch
import numpy as np

store_dir = 'store'
datasets_dir = 'vision_datasets'
export_dir = os.path.join(store_dir, 'models', 'exported')
if not os.path.exists(export_dir):
    os.makedirs(export_dir)

In [2]:
from blurnet import trainer

train_job = trainer.TrainJob(store_dir, datasets_dir)

In [3]:
task, grayscale = 'CIFAR10', True
export_pths = {}
sigmas = {
    'baseline': None,
    'blur': 1.5,
}

for label, sigma in sigmas.items():
    cond = {'model_config': {'task': task, 'grayscale': grayscale, 'sigma': sigma}}
    export_pths[label] = []
    for key, config in train_job.conditioned(cond):
        export_pth = os.path.join(export_dir, '{}-{}-{}-{:03d}.pickle'.format(
            task, 'GRAY' if grayscale else 'RGB',
            'baseline' if sigma is None else '{:g}'.format(sigma),
            config['train_config']['seed']
        ))
        if not os.path.exists(export_pth):
            train_job.export(key, export_pth)
        export_pths[label].append(export_pth)
    print('{} models exported for sigma={}'.format(len(export_pths[label]), sigma))

10 models exported for sigma=None
10 models exported for sigma=1.5


In [4]:
from roarena.attack import AttackJob

attack_job = AttackJob(store_dir, datasets_dir)

/foolbox/foolbox/attacks/brendel_bethge.py:624: UserWarning: At the first initialisation the optimizer needs to be compiled. This may take between 20 to 60 seconds.
  warnings.warn(


In [8]:
model_pths = export_pths['blur']
success_rates, _ = attack_job.summarize(model_pths, 'L2', False, 0.25)

results pooled for store/models/exported/CIFAR10-GRAY-1.5-006.pickle (4m28.25s)
results pooled for store/models/exported/CIFAR10-GRAY-1.5-002.pickle (3m03.23s)
results pooled for store/models/exported/CIFAR10-GRAY-1.5-008.pickle (2m35.51s)
results pooled for store/models/exported/CIFAR10-GRAY-1.5-000.pickle (2m50.80s)
results pooled for store/models/exported/CIFAR10-GRAY-1.5-007.pickle (4m34.07s)
results pooled for store/models/exported/CIFAR10-GRAY-1.5-001.pickle (4m13.81s)
results pooled for store/models/exported/CIFAR10-GRAY-1.5-005.pickle (5m09.39s)
results pooled for store/models/exported/CIFAR10-GRAY-1.5-003.pickle (5m48.78s)
results pooled for store/models/exported/CIFAR10-GRAY-1.5-009.pickle (3m07.29s)
results pooled for store/models/exported/CIFAR10-GRAY-1.5-004.pickle (6m13.20s)


In [9]:
success_rates

array([0.893, 0.89 , 0.895, 0.879, 0.901, 0.905, 0.905, 0.895, 0.906,
       0.895])

In [10]:
model_pths = export_pths['baseline'][:4]
success_rates, _ = attack_job.summarize(model_pths, 'L2', False, 0.25, 25)

results pooled for store/models/exported/CIFAR10-GRAY-baseline-006.pickle (0m46.40s)
results pooled for store/models/exported/CIFAR10-GRAY-baseline-000.pickle (0m32.26s)
results pooled for store/models/exported/CIFAR10-GRAY-baseline-009.pickle (0m29.52s)
results pooled for store/models/exported/CIFAR10-GRAY-baseline-004.pickle (0m34.04s)


In [11]:
success_rates

array([0.984, 0.99 , 0.986, 0.99 ])

In [4]:
from roarena.corruption import CorruptionJob

corruption_job = CorruptionJob(store_dir, datasets_dir)

In [5]:
accs

{'gaussian_noise': array([0.6138, 0.615 , 0.6477, 0.6535, 0.6383, 0.6494, 0.622 , 0.6387,
        0.6459, 0.6445]),
 'shot_noise': array([0.6286, 0.6352, 0.6719, 0.6768, 0.6543, 0.6627, 0.643 , 0.6589,
        0.6627, 0.6565]),
 'impulse_noise': array([0.3924, 0.4137, 0.4449, 0.4552, 0.4245, 0.4339, 0.419 , 0.449 ,
        0.4489, 0.4515]),
 'defocus_blur': array([0.4581, 0.5093, 0.5374, 0.5366, 0.502 , 0.5504, 0.5115, 0.5043,
        0.5164, 0.5651]),
 'glass_blur': array([0.5745, 0.5662, 0.5906, 0.5781, 0.577 , 0.5705, 0.5789, 0.5823,
        0.5853, 0.5878]),
 'motion_blur': array([0.4938, 0.522 , 0.5252, 0.5137, 0.5267, 0.537 , 0.5228, 0.5249,
        0.5143, 0.5256]),
 'zoom_blur': array([0.629 , 0.6589, 0.6717, 0.6816, 0.6499, 0.6822, 0.6545, 0.645 ,
        0.6567, 0.6818]),
 'snow': array([0.72  , 0.7154, 0.7216, 0.7357, 0.7261, 0.7272, 0.7165, 0.7142,
        0.7274, 0.727 ]),
 'frost': array([0.7189, 0.7108, 0.7382, 0.7356, 0.7184, 0.7304, 0.7199, 0.7194,
        0.7328, 0.73

In [13]:
model_pths = export_pths['baseline'][:4]
accs = corruption_job.summarize(model_pths)

In [14]:
accs

{'gaussian_noise': array([0.2338, 0.2302, 0.2512, 0.2081]),
 'shot_noise': array([0.2844, 0.2708, 0.2403]),
 'impulse_noise': array([0.2743, 0.3521, 0.2871]),
 'defocus_blur': array([0.3388, 0.3623, 0.3154, 0.4108]),
 'glass_blur': array([0.327 , 0.3348, 0.34  , 0.3298]),
 'motion_blur': array([0.4706, 0.4692, 0.465 , 0.4946]),
 'zoom_blur': array([0.3543, 0.4773]),
 'snow': array([0.6444, 0.6407, 0.637 ]),
 'frost': array([0.4984, 0.4911, 0.5129, 0.4597]),
 'fog': array([0.5785, 0.6144, 0.6203]),
 'brightness': array([0.8789, 0.869 , 0.877 , 0.8676]),
 'contrast': array([0.41  , 0.4562, 0.5243, 0.4455]),
 'elastic_transform': array([0.6154, 0.6052, 0.6094, 0.6098]),
 'pixelate': array([0.3034, 0.3063, 0.3228, 0.294 ]),
 'jpeg_compression': array([0.6778, 0.6788, 0.6923, 0.6921])}